<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Automated Scoring System

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Store the fight and fighter data into csv files
* Preprocess the data
* Train and evaluate a neural network to predict fight outcomes

## All bouts and all fighters

In [2]:
import pandas as pd

In [3]:
all_past_events_url = "http://ufcstats.com/statistics/events/completed?page=all"
all_past_events_tables = pd.read_html(all_past_events_url)[0] # Returns list of all tables on page

In [4]:
all_past_events_tables.head()

,Name/date,Location
0,NaN,NaN
1,"UFC Fight Night: Hall vs. Silva October 31, 2020","Las Vegas, Nevada, USA"
2,"UFC 254: Khabib vs. Gaethje October 24, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates"
3,UFC Fight Night: Ortega vs. The Korean Zombie ...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"
4,UFC Fight Night: Moraes vs. Sandhagen October...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"


In [5]:
from string import ascii_lowercase

all_fighters_tables = []
for c in ascii_lowercase:
  all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
  all_fighters_table = pd.read_html(all_fighters_url)[0]
  all_fighters_tables.append(all_fighters_table)

all_fighters_tables = pd.concat(all_fighters_tables)

In [6]:
all_fighters_tables.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN


## Getting the data for one fight

This will look at Brian Ortega vs. Chan Sung Jung.

UFC Fight Night: Ortega vs. The Korean Zombie

http://ufcstats.com/fight-details/5a440bdee19efaab

In [7]:
ortega_vs_jung_url = "http://ufcstats.com/fight-details/5a440bdee19efaab"
ortega_vs_jung_tables = pd.read_html(ortega_vs_jung_url)
print(f"This URL has {len(ortega_vs_jung_tables)} tables")

This URL has 4 tables


In [8]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [9]:
ortega_vs_jung_tables[1].head()

Fighter      KD  ...    Rev.        Ctrl
                       Round 1 Round 1  ... Round 1     Round 1
                       Round 2 Round 2  ... Round 2     Round 2
                       Round 3 Round 3  ... Round 3     Round 3
                       Round 4 Round 4  ... Round 4     Round 4
                       Round 5 Round 5  ... Round 5     Round 5
0  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:00  0:00
1  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:35  0:00
2  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00
3  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:22  0:00
4  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00

[5 rows x 10 columns]

In [10]:
ortega_vs_jung_tables[2].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Brian Ortega Chan Sung Jung,127 of 212 62 of 163,59% 38%,64 of 135 35 of 127,22 of 28 14 of 21,41 of 49 13 of 15,118 of 199 62 of 162,5 of 7 0 of 1,4 of 6 0 of 0


In [11]:
ortega_vs_jung_tables[3].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3
,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4
,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5
0,Brian Ortega Chan Sung Jung,24 of 39 14 of 26,61% 53%,11 of 24 4 of 15,5 of 6 4 of 5,8 of 9 6 of 6,24 of 39 14 of 26,0 of 0 0 of 0,0 of 0 0 of 0,NaN
1,Brian Ortega Chan Sung Jung,23 of 37 14 of 34,62% 41%,14 of 25 12 of 31,2 of 4 0 of 1,7 of 8 2 of 2,17 of 29 14 of 34,2 of 2 0 of 0,4 of 6 0 of 0,NaN
2,Brian Ortega Chan Sung Jung,28 of 42 7 of 32,66% 21%,13 of 25 2 of 23,4 of 4 1 of 3,11 of 13 4 of 6,27 of 41 7 of 32,1 of 1 0 of 0,0 of 0 0 of 0,NaN
3,Brian Ortega Chan Sung Jung,17 of 35 8 of 26,48% 30%,5 of 18 1 of 18,5 of 6 6 of 7,7 of 11 1 of 1,15 of 33 8 of 26,2 of 2 0 of 0,0 of 0 0 of 0,NaN
4,Brian Ortega Chan Sung Jung,35 of 59 19 of 45,59% 42%,21 of 43 16 of 40,6 of 8 3 of 5,8 of 8 0 of 0,35 of 57 19 of 44,0 of 2 0 of 1,0 of 0 0 of 0,NaN


Note how each cell actually has two datapoints, which is a bit frustrating. We need to break them up into different cells.

In [12]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [13]:
ortega_vs_jung_tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [14]:
for i, row in all_fighters_tables.iterrows():
    fighter_name = f'{row["First"]} {row["Last"]}'
    if ortega_vs_jung_tables[0]["Fighter"][0].startswith(fighter_name):
        first_fighter = fighter_name
        second_fighter = ortega_vs_jung_tables[0]["Fighter"][0][len(fighter_name)+1:]
        print(f"First fighter is {first_fighter}")
        print(f'Second fighter is {second_fighter}')
        break

First fighter is Brian Ortega
Second fighter is Chan Sung Jung


In [15]:
ortega_vs_jung_tables[0]["KD"][0]

'2  0'

In [16]:
kd_string_split = ortega_vs_jung_tables[0]["KD"][0].split(" ")
first_fighter_kd = " ".join(kd_string_split[:len(kd_string_split)//2])
second_fighter_kd = " ".join(kd_string_split[len(kd_string_split)//2+1:])
print(f"First fighter kds: {first_fighter_kd}")
print(f'Second fighter kds: {second_fighter_kd}')

First fighter kds: 2
Second fighter kds: 0


In [17]:
def parse_two_fighter_stats(fight_table, column):
    string_split = fight_table[column][0].split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [18]:
for column in ortega_vs_jung_tables[0].columns:
    if column == "Fighter":
        continue
    fighter_1_stat, fighter_2_stat = parse_two_fighter_stats(ortega_vs_jung_tables[0], column)
    print(f"{column} 1: {fighter_1_stat}")
    print(f"{column} 2: {fighter_2_stat}")
    print()

KD 1: 2
KD 2: 0

Sig. str. 1: 127 of 212
Sig. str. 2: 62 of 163

Sig. str. % 1: 59%
Sig. str. % 2: 38%

Total str. 1: 129 of 214
Total str. 2: 64 of 166

Td 1: 3 of 10
Td 2: 0 of 0

Td % 1: 30%
Td % 2: ---

Sub. att 1: 0
Sub. att 2: 0

Rev. 1: 0
Rev. 2: 0

Ctrl 1: 0:57
Ctrl 2: 0:00



Now do the same for other tables

In [19]:
ortega_vs_jung_tables[1]

Fighter      KD  ...    Rev.        Ctrl
                       Round 1 Round 1  ... Round 1     Round 1
                       Round 2 Round 2  ... Round 2     Round 2
                       Round 3 Round 3  ... Round 3     Round 3
                       Round 4 Round 4  ... Round 4     Round 4
                       Round 5 Round 5  ... Round 5     Round 5
0  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:00  0:00
1  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:35  0:00
2  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00
3  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:22  0:00
4  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00

[5 rows x 10 columns]

In [20]:
new_columns = ["Fighter 1", "Fighter 2"]
for i, column in enumerate(ortega_vs_jung_tables[1]):
    if i == 0:
        continue
    new_columns.append(f"Fighter 1 {column[0]}")
    new_columns.append(f"Fighter 2 {column[0]}")
print(new_columns)

['Fighter 1', 'Fighter 2', 'Fighter 1 KD', 'Fighter 2 KD', 'Fighter 1 Sig. str.', 'Fighter 2 Sig. str.', 'Fighter 1 Sig. str. %', 'Fighter 2 Sig. str. %', 'Fighter 1 Total str.', 'Fighter 2 Total str.', 'Fighter 1 Td %', 'Fighter 2 Td %', 'Fighter 1 Td %', 'Fighter 2 Td %', 'Fighter 1 Sub. att', 'Fighter 2 Sub. att', 'Fighter 1 Rev.', 'Fighter 2 Rev.', 'Fighter 1 Ctrl', 'Fighter 2 Ctrl']


In [21]:
def parse_string(row_string):
    string_split = row_string.split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [22]:
rows = [[] for _ in range(len(new_columns))]
for i, row in ortega_vs_jung_tables[1].iterrows():
    rows[0].append(first_fighter)
    rows[1].append(second_fighter)
    index = 2
    for column in ortega_vs_jung_tables[1]:
        if column[0] == "Fighter":
            continue
        stat1, stat2 = parse_string(row[column[0]][0])
        rows[index].append(stat1)
        rows[index+1].append(stat2)
        index += 2

In [23]:
print(rows)

[['Brian Ortega', 'Brian Ortega', 'Brian Ortega', 'Brian Ortega', 'Brian Ortega'], ['Chan Sung Jung', 'Chan Sung Jung', 'Chan Sung Jung', 'Chan Sung Jung', 'Chan Sung Jung'], ['1', '1', '0', '0', '0'], ['0', '0', '0', '0', '0'], ['24 of 39', '23 of 37', '28 of 42', '17 of 35', '35 of 59'], ['14 of 26', '14 of 34', '7 of 32', '8 of 26', '19 of 45'], ['61%', '62%', '66%', '48%', '59%'], ['53%', '41%', '21%', '30%', '42%'], ['24 of 39', '24 of 38', '28 of 42', '18 of 36', '35 of 59'], ['14 of 26', '14 of 34', '7 of 32', '10 of 29', '19 of 45'], ['0 of 0', '2 of 3', '0 of 1', '1 of 4', '0 of 2'], ['0 of 0', '0 of 0', '0 of 0', '0 of 0', '0 of 0'], ['0 of 0', '2 of 3', '0 of 1', '1 of 4', '0 of 2'], ['0 of 0', '0 of 0', '0 of 0', '0 of 0', '0 of 0'], ['0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0'], ['0:00', '0:35', '0:00', '0:22', '0:00'], ['0:00', '0:00', '0:00', '0:00', '0:00']]


In [24]:
new_rows = []
for i, row in ortega_vs_jung_tables[1].iterrows():
    new_row = []
    new_row.append(first_fighter)
    new_row.append(second_fighter)
    for column in ortega_vs_jung_tables[1]:
        if column[0] == "Fighter":
            continue
        stat1, stat2 = parse_string(row[column[0]][0])
        new_row.append(stat1)
        new_row.append(stat2)
    new_rows.append(new_row)

In [25]:
print(new_rows)

[['Brian Ortega', 'Chan Sung Jung', '1', '0', '24 of 39', '14 of 26', '61%', '53%', '24 of 39', '14 of 26', '0 of 0', '0 of 0', '0 of 0', '0 of 0', '0', '0', '0', '0', '0:00', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '1', '0', '23 of 37', '14 of 34', '62%', '41%', '24 of 38', '14 of 34', '2 of 3', '0 of 0', '2 of 3', '0 of 0', '0', '0', '0', '0', '0:35', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '28 of 42', '7 of 32', '66%', '21%', '28 of 42', '7 of 32', '0 of 1', '0 of 0', '0 of 1', '0 of 0', '0', '0', '0', '0', '0:00', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '17 of 35', '8 of 26', '48%', '30%', '18 of 36', '10 of 29', '1 of 4', '0 of 0', '1 of 4', '0 of 0', '0', '0', '0', '0', '0:22', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '35 of 59', '19 of 45', '59%', '42%', '35 of 59', '19 of 45', '0 of 2', '0 of 0', '0 of 2', '0 of 0', '0', '0', '0', '0', '0:00', '0:00']]


In [26]:
df = pd.DataFrame(new_rows, columns=new_columns) 

In [27]:
df.head()

,Fighter 1,Fighter 2,Fighter 1 KD,Fighter 2 KD,Fighter 1 Sig. str.,Fighter 2 Sig. str.,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Total str.,Fighter 2 Total str.,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Sub. att,Fighter 2 Sub. att,Fighter 1 Rev.,Fighter 2 Rev.,Fighter 1 Ctrl,Fighter 2 Ctrl
0,Brian Ortega,Chan Sung Jung,1,0,24 of 39,14 of 26,61%,53%,24 of 39,14 of 26,0 of 0,0 of 0,0 of 0,0 of 0,0,0,0,0,0:00,0:00
1,Brian Ortega,Chan Sung Jung,1,0,23 of 37,14 of 34,62%,41%,24 of 38,14 of 34,2 of 3,0 of 0,2 of 3,0 of 0,0,0,0,0,0:35,0:00
2,Brian Ortega,Chan Sung Jung,0,0,28 of 42,7 of 32,66%,21%,28 of 42,7 of 32,0 of 1,0 of 0,0 of 1,0 of 0,0,0,0,0,0:00,0:00
3,Brian Ortega,Chan Sung Jung,0,0,17 of 35,8 of 26,48%,30%,18 of 36,10 of 29,1 of 4,0 of 0,1 of 4,0 of 0,0,0,0,0,0:22,0:00
4,Brian Ortega,Chan Sung Jung,0,0,35 of 59,19 of 45,59%,42%,35 of 59,19 of 45,0 of 2,0 of 0,0 of 2,0 of 0,0,0,0,0,0:00,0:00


Do the same for the other table

In [36]:
new_columns = ["Fighter 1", "Fighter 2"]
for i, column in enumerate(ortega_vs_jung_tables[3]):
    if column[0] == "Fighter" or column[0] == "Unnamed: 9_level_0":
        continue
    new_columns.append(f"Fighter 1 {column[0]}")
    new_columns.append(f"Fighter 2 {column[0]}")
print(new_columns)

['Fighter 1', 'Fighter 2', 'Fighter 1 Sig. str', 'Fighter 2 Sig. str', 'Fighter 1 Sig. str. %', 'Fighter 2 Sig. str. %', 'Fighter 1 Head', 'Fighter 2 Head', 'Fighter 1 Body', 'Fighter 2 Body', 'Fighter 1 Leg', 'Fighter 2 Leg', 'Fighter 1 Distance', 'Fighter 2 Distance', 'Fighter 1 Clinch', 'Fighter 2 Clinch', 'Fighter 1 Ground', 'Fighter 2 Ground']


In [37]:
new_rows = []
for i, row in ortega_vs_jung_tables[3].iterrows():
    new_row = []
    new_row.append(first_fighter)
    new_row.append(second_fighter)
    for column in ortega_vs_jung_tables[3]:
        if column[0] == "Fighter" or column[0] == "Unnamed: 9_level_0":
            continue
        print(column[0])
        print(row["Sig. str"][0])
        print(row[column[0]][0])
        stat1, stat2 = parse_string(row[column[0]][0])
        new_row.append(stat1)
        new_row.append(stat2)
    new_rows.append(new_row)

Sig. str
24 of 39  14 of 26
24 of 39  14 of 26
Sig. str. %
24 of 39  14 of 26
61%  53%
Head
24 of 39  14 of 26
11 of 24  4 of 15
Body
24 of 39  14 of 26
5 of 6  4 of 5
Leg
24 of 39  14 of 26
8 of 9  6 of 6
Distance
24 of 39  14 of 26
24 of 39  14 of 26
Clinch
24 of 39  14 of 26
0 of 0  0 of 0
Ground
24 of 39  14 of 26
0 of 0  0 of 0
Sig. str
23 of 37  14 of 34
23 of 37  14 of 34
Sig. str. %
23 of 37  14 of 34
62%  41%
Head
23 of 37  14 of 34
14 of 25  12 of 31
Body
23 of 37  14 of 34
2 of 4  0 of 1
Leg
23 of 37  14 of 34
7 of 8  2 of 2
Distance
23 of 37  14 of 34
17 of 29  14 of 34
Clinch
23 of 37  14 of 34
2 of 2  0 of 0
Ground
23 of 37  14 of 34
4 of 6  0 of 0
Sig. str
28 of 42  7 of 32
28 of 42  7 of 32
Sig. str. %
28 of 42  7 of 32
66%  21%
Head
28 of 42  7 of 32
13 of 25  2 of 23
Body
28 of 42  7 of 32
4 of 4  1 of 3
Leg
28 of 42  7 of 32
11 of 13  4 of 6
Distance
28 of 42  7 of 32
27 of 41  7 of 32
Clinch
28 of 42  7 of 32
1 of 1  0 of 0
Ground
28 of 42  7 of 32
0 of 0  0 of 0
Si

In [38]:
df2 = pd.DataFrame(new_rows, columns=new_columns) 

In [39]:
df2.head()

,Fighter 1,Fighter 2,Fighter 1 Sig. str,Fighter 2 Sig. str,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Head,Fighter 2 Head,Fighter 1 Body,Fighter 2 Body,Fighter 1 Leg,Fighter 2 Leg,Fighter 1 Distance,Fighter 2 Distance,Fighter 1 Clinch,Fighter 2 Clinch,Fighter 1 Ground,Fighter 2 Ground
0,Brian Ortega,Chan Sung Jung,24 of 39,14 of 26,61%,53%,11 of 24,4 of 15,5 of 6,4 of 5,8 of 9,6 of 6,24 of 39,14 of 26,0 of 0,0 of 0,0 of 0,0 of 0
1,Brian Ortega,Chan Sung Jung,23 of 37,14 of 34,62%,41%,14 of 25,12 of 31,2 of 4,0 of 1,7 of 8,2 of 2,17 of 29,14 of 34,2 of 2,0 of 0,4 of 6,0 of 0
2,Brian Ortega,Chan Sung Jung,28 of 42,7 of 32,66%,21%,13 of 25,2 of 23,4 of 4,1 of 3,11 of 13,4 of 6,27 of 41,7 of 32,1 of 1,0 of 0,0 of 0,0 of 0
3,Brian Ortega,Chan Sung Jung,17 of 35,8 of 26,48%,30%,5 of 18,1 of 18,5 of 6,6 of 7,7 of 11,1 of 1,15 of 33,8 of 26,2 of 2,0 of 0,0 of 0,0 of 0
4,Brian Ortega,Chan Sung Jung,35 of 59,19 of 45,59%,42%,21 of 43,16 of 40,6 of 8,3 of 5,8 of 8,0 of 0,35 of 57,19 of 44,0 of 2,0 of 1,0 of 0,0 of 0


## Going to future pages

In [40]:
from urllib.request import urlopen

In [41]:
all_past_events_page = urlopen(all_past_events_url)

In [42]:
all_past_events_html_bytes = all_past_events_page.read()
all_past_events_html = all_past_events_html_bytes.decode("utf-8")

In [43]:
event1, event2 = all_past_events_tables["Name/date"][1], all_past_events_tables["Name/date"][2]

In [48]:
print(len(all_past_events_tables.index))

539


In [44]:
i1, i2 = all_past_events_html.find(event1[:event1.find("vs")]), all_past_events_html.find(event2[:event2.find("vs")])
event_string = all_past_events_html[i1:i2]
http_index = event_string.find('http://')
short = event_string[http_index:]
short = short[:short.find("\"")]

In [45]:
event_string

'UFC Fight Night: Hall vs. Silva\n                        </a>\n                        <span class="b-statistics__date">\n                          October 31, 2020\n                        </span>\n                    </i>\n                  </td>\n                  <td class="b-statistics__table-col b-statistics__table-col_style_big-top-padding">\n                    Las Vegas, Nevada, USA\n                  </td>\n                </tr>\n              \n                <tr class="b-statistics__table-row">\n                  <td class="b-statistics__table-col">\n                  \n                    <i class="b-statistics__table-content">\n                        <a href="http://ufcstats.com/event-details/c3c38c86f5ab9b5c" class="b-link b-link_style_black">\n                          '

In [46]:
short

'http://ufcstats.com/event-details/c3c38c86f5ab9b5c'

In [47]:
all_past_events_tables["Name/date"][2]

'UFC 254: Khabib vs. Gaethje  October 24, 2020'

In [57]:
def get_all_events(all_past_events_url):
    all_past_events_page = urlopen(all_past_events_url)
    all_past_events_html_bytes = all_past_events_page.read()
    all_past_events_html = all_past_events_html_bytes.decode("utf-8")
    all_past_events_tables = pd.read_html(all_past_events_url)[0]
    index = 2
    all_http_strings = []
    while index < len(all_past_events_tables.index):
        event1, event2 = all_past_events_tables["Name/date"][index-1], all_past_events_tables["Name/date"][index]

        i1, i2 = all_past_events_html.find(event1[:event1.find("vs")]), all_past_events_html.find(event2[:event2.find("vs")])
        event_string = all_past_events_html[i1:i2]
        http_index = event_string.find('http://')
        http_string = event_string[http_index:]
        http_string = http_string[:http_string.find("\"")]
        print(http_string)
        all_http_strings.append(http_string)
        index += 1
    return all_http_strings

all_event_http_strings = get_all_events(all_past_events_url)

http://ufcstats.com/event-details/c3c38c86f5ab9b5c
http://ufcstats.com/event-details/d4f364dd076bb0e2
http://ufcstats.com/event-details/fd4578cac86d75ca
http://ufcstats.com/event-details/805ad1801eb26abb
http://ufcstats.com/event-details/e69c5ce12f4e762b
http://ufcstats.com/event-details/831b937811804dad
http://ufcstats.com/event-details/9c37681096c6f3a9
http://ufcstats.com/event-details/542db012217ecb83
http://ufcstats.com/event-details/e29cf523ebd155c5
http://ufcstats.com/event-details/af1e5c64b8663aa0
http://ufcstats.com/event-details/bda04c573563cc2e
http://ufcstats.com/event-details/7a82635ffa9b59fe
http://ufcstats.com/event-details/3746e21bb508391a
http://ufcstats.com/event-details/dde70a112e053a6c
http://ufcstats.com/event-details/ddbd0d6259ce57cc
http://ufcstats.com/event-details/18f5669a92e99d92
http://ufcstats.com/event-details/dbd198f780286aca
http://ufcstats.com/event-details/c32eab6c2119e989
http://ufcstats.com/event-details/2eab7a6c8b0ed8cc
http://ufcstats.com/event-detai

In [61]:
def get_all_events(past_event_url):
    past_event_page = urlopen(past_event_url)
    past_event_html_bytes = past_event_page.read()
    past_event_html = past_event_html_bytes.decode("utf-8")
    past_event_tables = pd.read_html(past_event_url)[0]
    print(past_event_tables.head())
    return past_event_html
    # index = 2
    # all_http_strings = []
    # while index < len(all_past_events_tables.index):
    #     event1, event2 = all_past_events_tables["Name/date"][index-1], all_past_events_tables["Name/date"][index]

    #     i1, i2 = all_past_events_html.find(event1[:event1.find("vs")]), all_past_events_html.find(event2[:event2.find("vs")])
    #     event_string = all_past_events_html[i1:i2]
    #     http_index = event_string.find('http://')
    #     http_string = event_string[http_index:]
    #     http_string = http_string[:http_string.find("\"")]
    #     print(http_string)
    #     all_http_strings.append(http_string)
    #     index += 1
    # return all_http_strings

fight_http_strings = get_all_events(all_event_http_strings[0])

   W/L                              Fighter  ... Round  Time
0  win  Khabib Nurmagomedov  Justin Gaethje  ...     2  1:34
1  win    Robert Whittaker  Jared Cannonier  ...     3  5:00
2  win        Alexander Volkov  Walt Harris  ...     2  1:15
3  win         Phillip Hawes  Jacob Malkoun  ...     1  0:18
4  win      Lauren Murphy  Liliya Shakirova  ...     2  3:31

[5 rows x 10 columns]


In [62]:
fight_http_strings

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <title>\n    Stats | UFC\n  </title>\n  <meta name="description" content="">\n  <meta name="viewport" content="">\n  <link rel="stylesheet" href="/blocks/main.css?ver=902918">\n  <script src="/js/vendor/modernizr-2.6.2.min.js"></script>\n  <script>\n    (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n    })(window,document,\'script\',\'//www.google-analytics.com/analytics.js\',\'ga\');\n\n    ga(\'creat

In [64]:
body = fight_http_strings[fight_http_strings.index("<body "):]
before_win = body[:body.index("win")]
begin = before_win[before_win.rfind("http://"):]
summary = begin[:begin.find("\"")]
print(summary)

http://ufcstats.com/fight-details/59eea37af7efb06c


In [ ]:
<body class="b-page" data-link="home">
    <header class="b-head">
      <div class="b-head__nav-bar">
  <div class="l-page__container">
    <a class="b-head__logo" href="/">
        <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ufc-stats-logo.png"
             alt="">
    </a>
    <menu class="b-head__menu">
      
          <li class="b-head__item b-head__item_state_current">
            <a class="b-head__menu-link" href="/statistics/events/">
              EVENTS & FIGHTS
            </a>
          </li>
      
      
          <li class="b-head__item">
            <a class="b-head__menu-link" href="/statistics/fighters">
              FIGHTERS
            </a>
          </li>
      
      
          <li class="b-head__item">
            <a class="b-head__menu-link" href="http://statleaders.ufc.com" target="_blank">
              STAT LEADERS
            </a>
          </li>
      
    </menu>
  </div>
</div>
    <div class="b-head__sub-nav">
    </div>

    </header>
    <section class="b-statistics__section_details">
<script type="text/javascript">
    function doNav(url) {
        document.location.href = url;

    }
</script>
<div class="l-page__container">
<h2 class="b-content__title">

            <span class="b-content__title-highlight">
              UFC 254: Khabib vs. Gaethje
            </span>
</h2>

<div class="b-fight-details">
<div class="b-list__info-box b-list__info-box_style_large-width">
  <ul class="b-list__box-list">
    <li class="b-list__box-list-item">
      <i class="b-list__box-item-title">
        Date:
      </i>
      October 24, 2020
    </li>
    <li class="b-list__box-list-item">
      <i class="b-list__box-item-title">
        Location:
      </i>

      Abu Dhabi, Abu Dhabi, United Arab Emirates
    </li>
  </ul>
</div>

<div class="b-statistics__table-preview">
  <i class="b-statistics__table-preview-item">
    Click on a row below to see in-depth event stats.
  </i>
  <i class="b-statistics__table-preview-item">
    Fight, Perf, Sub, and KO of the Night Bonuses:
    <img class="b-statistics__table-status" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png" alt="">
    <img class="b-statistics__table-status" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png" alt="">
    <img class="b-statistics__table-status" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/sub.png" alt="">
    <img class="b-statistics__table-status" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ko.png" alt="">

  </i>
</div>

<table class="b-fight-details__table b-fight-details__table_style_margin-top b-fight-details__table_type_event-details js-fight-table" style="display: table;">
<thead class="b-fight-details__table-head">
<tr class="b-fight-details__table-row">
  <th class="b-fight-details__table-col">
    W/L
  </th>
  <th class="b-fight-details__table-col l-page_align_left">
    Fighter
  </th>
  <th class="b-fight-details__table-col">
    Kd
  </th>
  <th class="b-fight-details__table-col">
    Str
  </th>
  <th class="b-fight-details__table-col">
    Td
  </th>
  <th class="b-fight-details__table-col">
    Sub
  </th>
  <th class="b-fight-details__table-col l-page_align_left">
    Weight class
  </th>
  <th class="b-fight-details__table-col l-page_align_left">
    Method
  </th>
  <th class="b-fight-details__table-col">
    Round
  </th>
  <th class="b-fight-details__table-col">
    Time
  </th>
</tr>
</thead>
<tbody class="b-fight-details__table-body">



    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/59eea37af7efb06c')" data-link="http://ufcstats.com/fight-details/59eea37af7efb06c">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/59eea37af7efb06c"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/032cc3922d871c7f">
              Khabib Nurmagomedov
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/9e8f6c728eb01124">
              Justin Gaethje
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               26

          </p>

          <p class="b-fight-details__table-text">

            
            29
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            2
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Lightweight<br>
        <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/belt.png" alt=""><img style="margin-right:2px;" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png" alt="">
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        SUB

      </p>

      <p class="b-fight-details__table-text">
        Triangle Choke
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        2
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        1:34
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/51108eabae8391ca')" data-link="http://ufcstats.com/fight-details/51108eabae8391ca">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/51108eabae8391ca"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/e1147d3d2dabe1ce">
              Robert Whittaker
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/13a0275fa13c4d26">
              Jared Cannonier
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               69

          </p>

          <p class="b-fight-details__table-text">

            
            53
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Middleweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        U-DEC

      </p>

      <p class="b-fight-details__table-text">
        
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        3
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        5:00
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/ccfe7b0d29c5440c')" data-link="http://ufcstats.com/fight-details/ccfe7b0d29c5440c">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/ccfe7b0d29c5440c"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/279566840aa55bf2">
              Alexander Volkov
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/03f982ca3735070c">
              Walt Harris
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               45

          </p>

          <p class="b-fight-details__table-text">

            
            16
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Heavyweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        KO/TKO

      </p>

      <p class="b-fight-details__table-text">
        Kick
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        2
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        1:15
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/a2620cbef573a837')" data-link="http://ufcstats.com/fight-details/a2620cbef573a837">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/a2620cbef573a837"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/547afe1017e72dbe">
              Phillip Hawes
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/d4270315cbcf2569">
              Jacob Malkoun
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               7

          </p>

          <p class="b-fight-details__table-text">

            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Middleweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        KO/TKO

      </p>

      <p class="b-fight-details__table-text">
        Punches
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        0:18
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/e8ef17c0babcfca5')" data-link="http://ufcstats.com/fight-details/e8ef17c0babcfca5">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/e8ef17c0babcfca5"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/729d4bd5a6cd0a97">
              Lauren Murphy
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/f300763147320d12">
              Liliya Shakirova
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               22

          </p>

          <p class="b-fight-details__table-text">

            
            23
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Women's Flyweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        SUB

      </p>

      <p class="b-fight-details__table-text">
        Rear Naked Choke
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        2
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        3:31
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/eedcb6c5d163e581')" data-link="http://ufcstats.com/fight-details/eedcb6c5d163e581">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/eedcb6c5d163e581"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/d802174b0c0c1f4e">
              Magomed Ankalaev
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/cd13728ae1151f46">
              Ion Cutelaba
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            2
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               21

          </p>

          <p class="b-fight-details__table-text">

            
            11
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Light Heavyweight<br>
        <img style="margin-right:2px;" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png" alt="">
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        KO/TKO

      </p>

      <p class="b-fight-details__table-text">
        Punch
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        4:19
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/6eaa4981d0335c8a')" data-link="http://ufcstats.com/fight-details/6eaa4981d0335c8a">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/6eaa4981d0335c8a"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/c62fbc117d57b943">
              Tai Tuivasa
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/ebc1f40e00e0c481">
              Stefan Struve
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               27

          </p>

          <p class="b-fight-details__table-text">

            
            13
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Heavyweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        KO/TKO

      </p>

      <p class="b-fight-details__table-text">
        Punches
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        4:59
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/de2069fea664c4b7')" data-link="http://ufcstats.com/fight-details/de2069fea664c4b7">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/de2069fea664c4b7"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/e7bba7ce4d6c7b07">
              Casey Kenney
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/329e403448756217">
              Nathaniel Wood
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               123

          </p>

          <p class="b-fight-details__table-text">

            
            136
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            2
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Catch Weight<br>
        <img style="margin-right:2px;" src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png" alt="">
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        U-DEC

      </p>

      <p class="b-fight-details__table-text">
        
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        3
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        5:00
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/5b062be91c65fb73')" data-link="http://ufcstats.com/fight-details/5b062be91c65fb73">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/5b062be91c65fb73"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/01afe0916a40c7c5">
              Shavkat Rakhmonov
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/57b10fca4f9c9a7a">
              Alex Oliveira
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               13

          </p>

          <p class="b-fight-details__table-text">

            
            9
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Welterweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        SUB

      </p>

      <p class="b-fight-details__table-text">
        Guillotine Choke
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        4:40
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/55c257629ecea80f')" data-link="http://ufcstats.com/fight-details/55c257629ecea80f">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
                  <p class="b-fight-details__table-text">
                    <a class="b-flag b-flag_style_bordered" href="http://ufcstats.com/fight-details/55c257629ecea80f"><i class="b-flag__inner"><i class="b-flag__text">draw<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
                  </p>


                  <p class="b-fight-details__table-text">
                    <a class="b-flag b-flag_style_bordered" href="http://ufcstats.com/fight-details/55c257629ecea80f"><i class="b-flag__inner"><i class="b-flag__text">draw<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
                  </p>
              
          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/74f9448b97bf8bec">
              Da-Un Jung
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/d156513a19acf856">
              Sam Alvey
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            1
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               56

          </p>

          <p class="b-fight-details__table-text">

            
            59
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Light Heavyweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        S-DEC

      </p>

      <p class="b-fight-details__table-text">
        
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        3
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        5:00
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/47f9c14a7dca1ad7')" data-link="http://ufcstats.com/fight-details/47f9c14a7dca1ad7">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/47f9c14a7dca1ad7"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/f29a6350c69f4a33">
              Miranda Maverick
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/eebc23626e38f5dc">
              Liana Jojua
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               49

          </p>

          <p class="b-fight-details__table-text">

            
            30
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Women's Flyweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        KO/TKO

      </p>

      <p class="b-fight-details__table-text">
        
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        5:00
      </p>
    </td>
    </tr>


    

    <tr class="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click" onclick="doNav('http://ufcstats.com/fight-details/2279d909f6e3e552')" data-link="http://ufcstats.com/fight-details/2279d909f6e3e552">
    <td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">

      
          
              <p class="b-fight-details__table-text">
                <a class="b-flag b-flag_style_green" href="http://ufcstats.com/fight-details/2279d909f6e3e552"><i class="b-flag__inner"><i class="b-flag__text">win<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
              </p>
              <br>

          
      


    </td>
    <td style="width:100px" class="b-fight-details__table-col l-page_align_left">
      
          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/58bbef3770bb2dfc">
              Joel Alvarez
            </a>

          </p>

          <p class="b-fight-details__table-text">

            <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/d06f8869449ba7bd">
              Alexander Yakovlev
            </a>
          </p>

      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col">

      

             <p class="b-fight-details__table-text">
               
               4

          </p>

          <p class="b-fight-details__table-text">

            
            1
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            0
          </p>

          <p class="b-fight-details__table-text">
            
            1
          </p>
      
    </td>
    <td class="b-fight-details__table-col">
      

          <p class="b-fight-details__table-text">
            
            2
          </p>

          <p class="b-fight-details__table-text">
            
            0
          </p>
      
    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        Lightweight<br>
        
      </p>

    </td>
    <td class="b-fight-details__table-col l-page_align_left">
      <p class="b-fight-details__table-text">
        
        

        SUB

      </p>

      <p class="b-fight-details__table-text">
        Armbar
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">

        1
      </p>
    </td>
    <td class="b-fight-details__table-col">
      <p class="b-fight-details__table-text">
        3:00
      </p>
    </td>
    </tr>
</tbody>
</table>
</div>
</div>
</section>
    <footer class="b-foot">
  <div class="b-foot__inner">
    <div class="l-page__container clearfix">
      <div class="b-foot__right">
        <div class="b-foot__list-inner">
          <h4 class="b-foot__list-title">
            Navigation
          </h4>
          <ul class="b-foot__list">
            <li class="b-foot__item">
              <a class="b-foot__link" href="/statistics/events/">
                Events &amp; Fights
              </a>
            </li>
            <li class="b-foot__item">
              <a class="b-foot__link" href="/statistics/fighters">
                Fighters
              </a>
            </li>
            <li class="b-foot__item">
              <a class="b-foot__link" href="http://statleaders.ufc.com" target="_blank">
                Stat Leaders
              </a>
            </li>
          </ul>
        </div>     
      </div>
      <div class="b-foot__logo">
        <a href="/">
        <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ufc-stats-foot-logo.png" alt="">
        </a>
        <div class="b-foot__copy-inner">
          &copy; All Rights Reserved |
          <a class="b-foot__copy-link" href="https://ufc.com/terms" target="_blank">Terms of Use</a> |
          <a class="b-foot__copy-link" href="https://ufc.com/privacy-policy" target="_blank">Privacy Policy</a>
        </div>
      </div>      
    </div>
  </div>
</footer>
    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script src="http://code.jquery.com/jquery-1.9.1.js"></script>
<script src="http://code.jquery.com/ui/1.10.3/jquery-ui.js"></script>
<link rel="stylesheet" href="http://code.jquery.com/ui/1.10.3/themes/smoothness/jquery-ui.css" />
<script>window.jQuery || document.write('<script src="/js/vendor/jquery-1.10.2.min.js"><\/script>')</script>
<script type="text/javascript" src="/js/vendor/jquery.validate.min.js"></script>
<script src="/js/plugins.js"></script>
<script src="/js/main.js?ver=903265"></script>
<script type="text/javascript">
    $(document).ready(function () {
        $(function () {
            $("#dialog").dialog({

                autoOpen: false,
                modal: true,
                show: {
                    effect: "blind",
                    duration: 1000
                },
                hide: {
                    effect: "fold",
                    duration: 1000
                }

            });
        });
    }

    );
    $("#opener").click(function () {
        $("#dialog").dialog("open");
    });
    $("#opener").click(function () {
        var state = true;
        if (state) {
            $("#dialog").animate({
                backgroundColor: "white",
                color: "black"

            }, 1000);
        }
    });

</script>

<script type="text/javascript">

 /*window.onload = function (){
     // location.pathname.split( '/' )
     theUrl = window.location.host;


     if(theUrl === 'hosteddb.fightmetric.com'){
         window.location.replace("http://www.fightmetric.com/statistics/events/upcoming");
     }

     //alert(theUrl);
  }*/
  //window.onpaint = handleHosted;
</script>
</body>
</html>